In [11]:
# 导入必要的库
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

In [ ]:
# 定义一个函数来显示图像
def show_image(image, title='Image'):
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# 定义一个函数来绘制直方图
def plot_histogram(image, bins=256):
    plt.hist(image.ravel(), bins=bins, range=[0, 256], color='r', alpha=0.7)
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.title('Histogram')
    plt.show()

# 定义一个函数来处理图像
def process_image(image, method, sub_option, threshold=None):
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    processed_image = None
    
    if method == '灰度化':
        if sub_option == '绘制灰度直方图':
            plot_histogram(gray_image)
            processed_image = gray_image  # 返回灰度图像
        elif sub_option == '灰度直方图修正':
            processed_image = cv.equalizeHist(gray_image)
        elif sub_option == '线性变换':
            alpha = 1.5  # 对比度增益
            beta = 15    # 亮度增益
            processed_image = cv.convertScaleAbs(gray_image, alpha=alpha, beta=beta)
        elif sub_option == '对数变换':
            c = 1
            log_transformed_image = c * np.log(1 + gray_image).astype(np.uint8)
            processed_image = (log_transformed_image / np.max(log_transformed_image) * 255).astype(np.uint8)
        elif sub_option == '指数变换':
            c = 1
            gamma = 0.5
            exp_transformed_image = c * np.power(gray_image, gamma).astype(np.uint8)
            processed_image = (exp_transformed_image / np.max(exp_transformed_image) * 255).astype(np.uint8)
    elif method == '二值化':
        if sub_option == '固定阈值分割':
            _, processed_image = cv.threshold(gray_image, threshold, 255, cv.THRESH_BINARY)
        elif sub_option == '自适应均值阈值分割':
            processed_image = cv.adaptiveThreshold(gray_image, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 11, 5)
        elif sub_option == '自适应高斯阈值分割':
            processed_image = cv.adaptiveThreshold(gray_image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 5)
        elif sub_option == 'Otsu自动计算阈值分割':
            _, processed_image = cv.threshold(gray_image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    return processed_image

In [13]:
# 创建文件上传控件
file_upload = widgets.FileUpload(
    accept='image/*',  # 接受所有图片格式
    multiple=False  # 不允许多文件上传
)

# 创建方法选择下拉菜单
method_dropdown = widgets.Dropdown(
    options=['灰度化', '二值化'],
    value='灰度化',
    description='处理方法:',
)

# 创建子选项选择下拉菜单
sub_option_dropdown = widgets.Dropdown(
    options=['绘制灰度直方图', '灰度直方图修正', '线性变换', '对数变换', '指数变换'],
    value='绘制灰度直方图',
    description='子选项:',
)

# 创建阈值滑动条，并默认隐藏
threshold_slider = widgets.IntSlider(
    min=0,
    max=255,
    step=1,
    value=127,
    description='阈值:',
    layout={'visibility': 'hidden'}  # 默认隐藏
)

# 创建处理按钮
process_button = widgets.Button(
    description='处理',
)

# 创建输出区域
output = widgets.Output()

# 当文件上传后，读取图片并显示原始图片
def on_file_upload_changed(change):
    global image  # 使用全局变量存储图片数据
    uploaded_files = change['new']
    if isinstance(uploaded_files, tuple):  # 如果上传了多个文件，则转换为字典形式
        uploaded_files = {file.name: {'content': file.content} for file in uploaded_files}
    
    for filename, file_info in uploaded_files.items():
        content = file_info['content']
        image = cv.imdecode(np.frombuffer(content, np.uint8), -1)
        with output:
            clear_output(wait=True)
            show_image(image, '原始图像')

file_upload.observe(on_file_upload_changed, names='value')

# 当处理按钮被点击时，处理图片
def on_process_button_clicked(b):
    with output:
        clear_output(wait=True)
        if method_dropdown.value == '二值化' and sub_option_dropdown.value == '固定阈值分割':
            processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value, threshold_slider.value)
        else:
            processed_image = process_image(image, method_dropdown.value, sub_option_dropdown.value)
        
        # 显示原始图像和处理后的图像
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        axs[0].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        axs[0].set_title('原始图像')
        axs[0].axis('off')
        
        axs[1].imshow(cv.cvtColor(processed_image, cv.COLOR_BGR2RGB) if len(processed_image.shape) == 3 else processed_image, cmap='gray')
        axs[1].set_title('处理后的图像')
        axs[1].axis('off')
        
        plt.show()


process_button.on_click(on_process_button_clicked)

# 更新子选项下拉列表
def on_method_change(change):
    if change['new'] == '灰度化':
        sub_option_dropdown.options = ['绘制灰度直方图', '灰度直方图修正', '线性变换', '对数变换', '指数变换']
        threshold_slider.layout.visibility = 'hidden'
    elif change['new'] == '二值化':
        sub_option_dropdown.options = ['固定阈值分割', '自适应均值阈值分割', '自适应高斯阈值分割', 'Otsu自动计算阈值分割']
        if sub_option_dropdown.value == '固定阈值分割':
            threshold_slider.layout.visibility = 'visible'
        else:
            threshold_slider.layout.visibility = 'hidden'

method_dropdown.observe(on_method_change, names='value')

# 当子选项改变时，检查是否需要显示阈值滑动条
def on_sub_option_change(change):
    if method_dropdown.value == '二值化' and change['new'] == '固定阈值分割':
        threshold_slider.layout.visibility = 'visible'
    else:
        threshold_slider.layout.visibility = 'hidden'

sub_option_dropdown.observe(on_sub_option_change, names='value')

# 显示控件
display(file_upload, method_dropdown, sub_option_dropdown, threshold_slider, process_button, output)

FileUpload(value=(), accept='image/*', description='Upload')

Dropdown(description='处理方法:', options=('灰度化', '二值化'), value='灰度化')

Dropdown(description='子选项:', options=('绘制灰度直方图', '灰度直方图修正', '线性变换', '对数变换', '指数变换'), value='绘制灰度直方图')

IntSlider(value=127, description='阈值:', layout=Layout(visibility='hidden'), max=255)

Button(description='处理', style=ButtonStyle())

Output()